## Imports and Functions

In [ ]:
!pip install igorwriter

In [ ]:
from igorwriter import IgorWave
from typing import List, Optional, Union
import re

import numpy as np
import matplotlib.pyplot as plt
# plt.style.use('ggplot')

from scipy.ndimage import gaussian_filter

import os
import pandas as pd

import sys
sys.path.append(r'C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python')
from arpes_functions import analysis_functions, plotting_functions, HDF5_loader, misc_functions, kw_data_loader
from arpes_functions import filter_functions as ff

In [ ]:
def save_to_igor_itx(file_path: str, xs: List[np.ndarray], datas: List[np.ndarray], names: List[str],
                     ys: Optional[List[np.ndarray]] = None,
                     x_labels: Optional[Union[str, List[str]]] = None,
                     y_labels: Optional[Union[str, List[str]]] = None):
    """Save data to a .itx file which can be dropped into Igor"""

    def check_axis_linear(arr: np.ndarray, axis: str, name: str, current_waves: list) -> bool:
        if arr.shape[-1] > 1 and not np.all(np.isclose(np.diff(arr), np.diff(arr)[0])):
            logger.warning(f"{file_path}: Igor doesn't support a non-linear {axis}-axis. Saving as separate wave")
            axis_wave = IgorWave(arr, name=name + f'_{axis}')
            current_waves.append(axis_wave)
            return False
        else:
            return True

    if x_labels is None or isinstance(x_labels, str):
        x_labels = [x_labels] * len(datas)
    if y_labels is None or isinstance(y_labels, str):
        y_labels = [y_labels] * len(datas)
    if ys is None:
        ys = [None] * len(datas)
    assert all([len(datas) == len(list_) for list_ in [xs, names, x_labels, y_labels]])

    waves = []
    for x, y, data, name, x_label, y_label in zip(xs, ys, datas, names, x_labels, y_labels):
        wave = IgorWave(data, name=name)
        if x is not None:
            if check_axis_linear(x, 'x', name, waves):
                wave.set_dimscale('x', x[0], np.mean(np.diff(x)), units=x_label)
        if y is not None:
            if check_axis_linear(y, 'y', name, waves):
                wave.set_dimscale('y', y[0], np.mean(np.diff(y)), units=y_label)
        elif y_label is not None:
            wave.set_datascale(y_label)
        waves.append(wave)

    with open(file_path, 'w') as fp:
        for wave in waves:
            wave.save_itx(fp, image=True)  # Image = True hopefully makes np and igor match in x/y

In [ ]:
def fix_itx_format(filename: str = 'test.itx'):
    with open(filename, 'r') as f:
        lines = f.readlines()
    #     for i in range(4):
    #         lines.append(f.readline())

    for ln in [1, -3, -2, -1]:
        lines[ln] = lines[ln].replace("'", "")


    new = lines[:-6]
    new[1] = 'WAVES/D/N='+lines[1][12:]
    new.append('END\n')

    dscale = lines[-3][2:-1].replace(',', ' ', 1)
    xscale = lines[-2][2:-1].replace(',', ' ', 1).replace(' ', '', 1)
    yscale = lines[-1][2:-1].replace(',', ' ', 1).replace(' ', '', 1)

    new.append(f'X {xscale}; {yscale}; {dscale}'.replace('"",', '"", '))

    with open('test_fixed.itx', 'w') as f:
        f.writelines(new)

In [ ]:
def load_denoised_data(file_path: str, filename: str):
    data_path = os.path.join(file_path, filename)
    df = pd.read_csv(data_path, skiprows=3, header=None, sep='\s+', skipfooter=2)
    data = np.array(df).T
    with open(data_path, 'r') as f:
        lines = f.readlines()
    last = lines[-1]

    x_start, x_step = [float(v) for v in re.search('x\s*(-?\d*.\d+),\s*(-?\d+.\d+)', last).groups()]
    y_start, y_step = [float(v) for v in re.search('y\s*(-?\d*.\d+),\s*(-?\d+.\d+)', last).groups()]

    x = np.linspace(x_start, x_start+data.shape[1]*x_step, data.shape[1])
    y = np.linspace(y_start, y_start+data.shape[0]*y_step, data.shape[0])
    return x, y, data    

# XUV Diffraction Data

In [ ]:
fp= r'E:\atully\k-corrected data\Apr_2021\XUV_FS_gamma0'
fn = r'XUV_FS_gamma0_gkw11_filteredFFT_0.00int.h5'
data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

In [ ]:
slice_dim='y'
int_range=0.2

for i, v in enumerate([18.3, 18.1, 17.9]):
    
    x, y, data2D = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=v, int_range=int_range)
    x_new, y_new, data_new = analysis_functions.limit_dataset(x=x, y=y, data=data2D, xlim=(-0.73, 0.52), ylim=(-1.4, 0.1))
    data_new = analysis_functions.norm_data(data_new)
    
    fig, ax = plotting_functions.plot_2D_mpl(x=x_new, y=y_new, data=data_new, cmap='Blues')
    
    plotting_functions.format_FS_plot(fig, ax, slice_val=v, EF=18.3)

#     if i == 0:
#         plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\FS\xuv_FS_EF.png')
#     if i == 1:
#         plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\FS\xuv_FS_-0.2.png')
#     if i == 2:
#         plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\FS\xuv_FS_-0.4.png')

In [ ]:
#     plt.xticks(np.arange(-0.6, 0.52, 0.3), fontsize=14)
#     ax.set_xlabel(xlabel='kx (A-1)', fontsize=16)
#     plt.yticks(np.arange(-1.2, 0.2, 0.4), fontsize=14)
#     ax.set_ylabel(ylabel='ky (A-1)', fontsize=16)
#     ax.set_title(f'{np.round((-18.3+v), 2)} eV', fontsize=18)

#     ratio = 1.0
#     x_left, x_right = ax.get_xlim()
#     y_low, y_high = ax.get_ylim()
#     ax.set_aspect(abs((x_right-x_left)/(y_low-y_high))*ratio)

In [ ]:
# Get Raw Dataslice

# set slice parameters
slice_dim='y'
int_range=0.2
slice_val=18.3
# slice_val=18.1
# slice_val=17.9

# analysis -- get slice, FFT, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(x=xaxis, y=yaxis, data=dataslice, xlim=(-0.73, 0.52), ylim=(-1.4, 0.1))
d = analysis_functions.norm_data(d)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=d, xlabel='kx', ylabel='ky', title=f'Ek={slice_val}', cmap='Blues')

plotting_functions.format_FS_plot(fig, ax, slice_val=slice_val, EF=18.3)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\xuv_FS_EF.png')

# # Save to .itx (igor) file
# save_to_igor_itx('test.itx', [x], [d], ["cones"], [y])  # take every other y value to make dataset smaller
# fix_itx_format('test.itx')  # fix itx formatting for denoising website

In [ ]:
# Enhance Image
path = r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data'
old_file = r'xuv_FS_EF.png'
new_file = r'xuv_FS_EF_enhanced.png'
ff.enhance_image(path, old_file, new_file, factor=2)

In [ ]:
# Gaussian Mask
gauss_data = gaussian_filter(d, sigma=2)
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=gauss_data, xlabel='kx', ylabel='ky', title=f'Gaussian Mask: Ek={slice_val}', cmap='Blues')

format_FS_plot(fig, ax, EF=18.3)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\FS\xuv_FS_EF_gauss.png')
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\xuv_FS_EF_gauss.png')

In [ ]:
# Denoised Data
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\FS'
fn_dn = 'XUV_FS_EF_dn.itx'

title = re.search('FS_(.*?)\_', fn_dn).group()
title = title[3:-1]

x, y, data_dn = load_denoised_data(fp_dn, fn_dn)

d = analysis_functions.norm_data(data_dn)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=d, xlabel='kx', ylabel='ky', cmap='Blues')

plotting_functions.format_FS_plot(fig, ax, EF=18.3, title=title)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\FS\xuv_FS_EF_dn.png')
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\xuv_FS_EF_dn.png')

In [ ]:
# test = re.search('FS_(.*?)\_', fn).group()
# # test = re.search('(?<=\FS_)(.*?)(?=\_)', fn).group()
# test[3:-1]
# # 'x\s*(-?\d*.\d+)'
# # (?<=\[)(.*?)(?=\])
# # \[(.*?)\]

## XUV ky=-0.36 Cones

In [ ]:
# Get Raw Dataslice

# set slice parameters
slice_dim='z'
int_range=0.1
slice_val=-0.36

# analysis -- get slice of FFT_filtered dataset, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(17.55, 18.4))
# x, y, d = limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.4, 0.4), ylim=(17.55, 18.4))
normed_data = analysis_functions.norm_data(d)

# plot slice
# fig, ax = plotting_functions.plot_2D_mpl(x=xaxis, y=yaxis, data=dataslice, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)

plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\xuv_ky=-0.36\xuv_ky=-0.36.png')
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\xuv_ky=-0.36.png')

# # Save to .itx (igor) file
# # save_to_igor_itx('test.itx', [x], [d[::2]], ["cones"], [y[::2]])  # take every other y value to make dataset smaller
# save_to_igor_itx('test.itx', [x], [d], ["cones"], [y])  # take every other y value to make dataset smaller
# fix_itx_format('test.itx')  # fix itx formatting for denoising website

In [ ]:
# Enhance Image
path = r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data'
old_file = r'xuv_ky=-0.36.png'
new_file = r'xuv_ky=-0.36_enhanced.png'
ff.enhance_image(path, old_file, new_file, factor=2)

In [ ]:
# Gaussian Mask
gauss_data = gaussian_filter(d, sigma=2)
normed_data=analysis_functions.norm_data(gauss_data)
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=gauss_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky', title=f'Gaussian Mask: ky={slice_val}')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\xuv_ky=-0.36_gauss.png')

In [ ]:
# Denoised Data
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\xuv_ky=-0.36'
fn_dn = 'xuv_ky=-0.36_dn.itx'

# title = re.search('FS_(.*?)\_', fn_dn).group()
# title = title[3:-1]
title = 'ky=-0.36'

x_dn, y_dn, data_dn = load_denoised_data(fp_dn, fn_dn)

d_dn = analysis_functions.norm_data(data_dn)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=18.3)

# plotting_functions.format_FS_plot(fig, ax, EF=18.3, title=title)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky', title=title)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\xuv_ky=-0.36_dn.png')

## XUV ky=-0.72 Cones

In [ ]:
# set slice parameters
slice_dim='z'
int_range=0.1
slice_val=-0.72

# analysis -- get slice of FFT_filtered dataset, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(17.55, 18.4))
normed_data = analysis_functions.norm_data(d)

# plot slice
# fig, ax = plotting_functions.plot_2D_mpl(x=xaxis, y=yaxis, data=dataslice, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)

plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\xuv_ky=-0.72\xuv_ky=-0.72.png')
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\xuv_ky=-0.72.png')

# # Save to .itx (igor) file
# save_to_igor_itx('test.itx', [x], [d], ["cones"], [y])  # take every other y value to make dataset smaller
# fix_itx_format('test.itx')  # fix itx formatting for denoising website

In [ ]:
# Enhance Image
path = r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data'
old_file = r'xuv_ky=-0.72.png'
new_file = r'xuv_ky=-0.72_enhanced.png'
ff.enhance_image(path, old_file, new_file, factor=2)

In [ ]:
# Gaussian Mask
gauss_data = gaussian_filter(d, sigma=2)
normed_data=analysis_functions.norm_data(gauss_data)
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=gauss_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky', title=f'Gaussian Mask: ky={slice_val}')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\xuv_ky=-0.72_gauss.png')

In [ ]:
# Denoised Data
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\xuv_ky=-0.72'
fn_dn = 'xuv_ky=-0.72_dn.itx'

# title = re.search('FS_(.*?)\_', fn_dn).group()
# title = title[3:-1]
title = 'ky=-0.72'

x_dn, y_dn, data_dn = load_denoised_data(fp_dn, fn_dn)

d_dn = analysis_functions.norm_data(data_dn)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=18.3)

# plotting_functions.format_FS_plot(fig, ax, EF=18.3, title=title)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky', title=title)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\xuv_ky=-0.72_dn.png')

# Lamp Diffraction Data

In [ ]:
fp = r'E:\atully\k-corrected data\Apr_2021\phi_motor_scan_FermiEdge2'
fn = r'phi_FermiEdge2kw_filteredFFT_0.00int.h5'
data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

In [ ]:
# set slice parameters
slice_dim='y'
int_range=0.2
slice_val=16.8

# analysis -- get slice, FFT, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.65, 0.52), ylim=(-1.4, 0.1))
normed_data = analysis_functions.norm_data(d)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='ky', title=f'Ek={slice_val}', cmap='Blues')

plotting_functions.format_FS_plot(fig, ax, slice_val=slice_val, EF=16.8)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Lamp diffraction data\Ek=16.8.png')
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\lamp_FS_EF.png')

# # Save to .itx (igor) file
# save_to_igor_itx('test.itx', [x], [d], ["cones"], [y])  # take every other y value to make dataset smaller
# fix_itx_format('test.itx')  # fix itx formatting for denoising website

In [ ]:
# Enhance Image
path = r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data'
old_file = r'lamp_FS_EF.png'
new_file = r'lamp_FS_EF_enhanced.png'
ff.enhance_image(path, old_file, new_file, factor=2)

In [ ]:
# Gaussian Mask
gauss_data = gaussian_filter(d, sigma=2)
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=gauss_data, xlabel='kx', ylabel='ky', title=f'Gaussian Mask: Ek={slice_val}', cmap='Blues')
plotting_functions.format_FS_plot(fig, ax, slice_val=slice_val, EF=16.8)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\lamp_FS_EF_gauss.png')

In [ ]:
# Denoised Data
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Lamp diffraction data\FS'
fn_dn = 'Ek=16.8_dn.itx'

# title = re.search('FS_(.*?)\_', fn_dn).group()
# title = title[3:-1]
title='EF'

x, y, data_dn = load_denoised_data(fp_dn, fn_dn)

d = analysis_functions.norm_data(data_dn)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=d, xlabel='kx', ylabel='ky', cmap='Blues')

plotting_functions.format_FS_plot(fig, ax, EF=16.8, title=title)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\lamp_FS_EF_dn.png')

## Lamp Diffraction Data -- Attempt at seeing cones

In [ ]:
# Take slices in ky

# set slice parameters
EF = 16.8
slice_dim='z'
int_range=0.1
# slice_val=-1.1
# slice_val=-0.4
# slice_val=-0.36
# slice_val=-0.8
slice_val=0

# analysis -- get slice of FFT_filtered dataset, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
# x, y, d = limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(15.6, 16.8))
# x, y, d = limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(15.8, 16.8))
x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(16.0, 16.8))
normed_data = analysis_functions.norm_data(d)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=EF)

plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Lamp diffraction data\ky=-0.4.png')  # this was with int_range = 0.0
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Lamp diffraction data\ky=-0.4_ylim=15.8to16.8.png')  # this was with int_range = 0.0

# # Save to .itx (igor) file
# save_to_igor_itx('test.itx', [x], [d], ["cones"], [y])  # take every other y value to make dataset smaller
# fix_itx_format('test.itx')  # fix itx formatting for denoising website

In [ ]:
# NOTE INT RANGE!

# set slice parameters
EF = 16.8
slice_dim='z'
int_range=0.05
# slice_val=-0.71
slice_val=-0.72

# analysis -- get slice of FFT_filtered dataset, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
# x, y, d = limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(15.6, 16.8))
x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(16.0, 16.8))
normed_data = analysis_functions.norm_data(d)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=EF)

plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Lamp diffraction data\ky=-0.71.png')  # int range 0.05 
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\lamp_ky=-0.71.png')  # int range 0.05 
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Lamp diffraction data\ky=-0.72.png')  # int range 0.05 
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\lamp_ky=-0.72.png')  # int range 0.05 

# # Save to .itx (igor) file
# save_to_igor_itx('test.itx', [x], [d], ["cones"], [y])  # take every other y value to make dataset smaller
# fix_itx_format('test.itx')  # fix itx formatting for denoising website

In [ ]:
# Enhance Image
path = r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data'
# old_file = r'lamp_ky=-0.71.png'
# new_file = r'lamp_ky=-0.71_enhanced.png'
old_file = r'lamp_ky=-0.72.png'
new_file = r'lamp_ky=-0.72_enhanced.png'
ff.enhance_image(path, old_file, new_file, factor=2)

In [ ]:
# Gaussian Mask
gauss_data = gaussian_filter(d, sigma=2)
normed_data=analysis_functions.norm_data(gauss_data)
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=gauss_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=16.8)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=16.8, slice_constant='ky', title=f'Gaussian Mask: ky={slice_val}')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\lamp_ky=-0.71_gauss.png')
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\lamp_ky=-0.72_gauss.png')

In [ ]:
# Denoised Data
# fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Lamp diffraction data\ky=-0.71'
# fn_dn = 'ky=-0.71_dn.itx'
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Lamp diffraction data\ky=-0.72'
fn_dn = 'ky=-0.72_dn.itx'

# title = re.search('FS_(.*?)\_', fn_dn).group()
# title = title[3:-1]
# title = 'ky=-0.71'
title = 'ky=-0.72'

x_dn, y_dn, data_dn = load_denoised_data(fp_dn, fn_dn)

d_dn = analysis_functions.norm_data(data_dn)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=16.8)

# plotting_functions.format_FS_plot(fig, ax, EF=18.3, title=title)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=16.8, slice_constant='ky', title=title)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\lamp_ky=-0.71_dn.png')
plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\lamp_ky=-0.72_dn.png')

# 5-6 ML Film: HOMO

In [ ]:
fp = r'E:\atully\arpes_data\2022_April\ARPES\C60\k_corrected'
fn = r'FS2andFS4_avgkw_filteredFFT_0.00int.h5'
data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

In [ ]:
# set slice parameters
EF = 18.48
slice_dim='x'
int_range=0.1
v = 0

# analysis -- get slice, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=v, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(x=yaxis, y=xaxis, data=dataslice.T, xlim=(-1.3, 0.18), ylim=(-2.7,-1.6), EF=EF)
d = analysis_functions.norm_data(d)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=d, EF=EF, title=f'HOMO: kx=0', xlabel='ky (A-1)', cmap='Blues', swap_axes=False)

# set aspect ratio
ratio = 1.0
x_left, x_right = ax.get_xlim()
y_low, y_high = ax.get_ylim()
ax.set_aspect(abs((x_right-x_left)/(y_low-y_high))*ratio)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\5-6 ML data\HOMO\xuv_kx=0_HOMO.png')

# # Save to .itx (igor) file
# save_to_igor_itx('test.itx', [x], [d], ["cones"], [y])  # take every other y value to make dataset smaller
# fix_itx_format('test.itx')  # fix itx formatting for denoising website

In [ ]:
slice_dim='y'
int_range=0.1

for i, v in enumerate([16.8, 16.4, 16.0]):
    
    x, y, data2D = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=v, int_range=int_range)
    x_new, y_new, data_new = analysis_functions.limit_dataset(x=x, y=y, data=data2D, xlim=(-0.52, 0.52), ylim=(-1.25, 0.1))
    data_new = analysis_functions.norm_data(data_new)
    
    fig, ax = plotting_functions.plot_2D_mpl(x=x_new, y=y_new, data=data_new, title=f'Constant Energy Slice: {v} eV', cmap='Blues')
    
    plotting_functions.format_FS_plot(fig, ax, EF=18.48, title=f'{np.round((-EF+v), 1)} eV', xticks=np.arange(-0.5, 0.52, 0.5))

#     if i == 0:
#         plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\5-6 ML data\HOMO\FS\xuv_FS_-1.7.png')
#     if i == 1:
#         plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\5-6 ML data\HOMO\FS\xuv_FS_-2.1.png')
#     if i == 2:
#         plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\5-6 ML data\HOMO\FS\xuv_FS_-2.5.png')

In [ ]:
# set slice parameters
EF = 18.48
slice_dim='y'
int_range=0.1
# v = 16.8
# v = 16.4
v = 16.0


xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=v, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(x=xaxis, y=yaxis, data=dataslice, xlim=(-0.52, 0.52), ylim=(-1.25, 0.1))
d = analysis_functions.norm_data(d)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=d, cmap='Blues', swap_axes=False)

plotting_functions.format_FS_plot(fig, ax, EF=18.48, title=f'{np.round((-EF+v), 1)} eV', xticks=np.arange(-0.5, 0.52, 0.5))

# # Save to .itx (igor) file
# save_to_igor_itx('test.itx', [x], [d], ["cones"], [y])  # take every other y value to make dataset smaller
# fix_itx_format('test.itx')  # fix itx formatting for denoising website

# 5-6 ML Film: HOMO-1

In [ ]:
fp = r'E:\atully\arpes_data\2022_April\ARPES\C60\k_corrected'
fn = r'FS3kw_filteredFFT_0.00int.h5'
data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

In [ ]:
# set slice parameters
EF = 18.48
slice_dim='x'
int_range=0.1
v = 0

# analysis -- get slice, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=v, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(x=yaxis, y=xaxis, data=dataslice.T, xlim=(-1.3, 0.18), ylim=(-3.9, -2.7), EF=EF)
d = analysis_functions.norm_data(d)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=d, EF=EF, title=f'HOMO-1: kx=0', xlabel='ky (A-1)', cmap='Blues', swap_axes=False)

# set aspect ratio
ratio = 1.0
x_left, x_right = ax.get_xlim()
y_low, y_high = ax.get_ylim()
ax.set_aspect(abs((x_right-x_left)/(y_low-y_high))*ratio)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\5-6 ML data\HOMO-1\xuv_kx=0_HOMO-1.png')

# # Save to .itx (igor) file
# save_to_igor_itx('test.itx', [x], [d], ["cones"], [y])  # take every other y value to make dataset smaller
# fix_itx_format('test.itx')  # fix itx formatting for denoising website

# Au Data

In [ ]:
slice_val = 16.8
int_range = 0.1

fp = r'E:\atully\arpes_data\2022_October\k_corrected'
fn = r'FS1_avg_gkw_filteredFFT_0.00int.h5'
data1, theta1, phi1, energy1 = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

# Get slice
xaxis1, yaxis1, dataslice1 = analysis_functions.get_2Dslice(x=theta1, y=energy1, z=phi1, data=data1, slice_dim='y', slice_val=slice_val, int_range=int_range)
dataslice1 = analysis_functions.norm_data(dataslice1)

fn = r'FS4_avg4_gkw_filteredFFT_0.00int.h5'
data4, theta4, phi4, energy4 = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

# Get slice
xaxis4, yaxis4, dataslice4 = analysis_functions.get_2Dslice(x=theta4, y=energy4, z=phi4, data=data4, slice_dim='y', slice_val=slice_val, int_range=int_range)
dataslice4 = analysis_functions.norm_data(dataslice4)

# 2

from scipy.interpolate import interp1d

new_theta = np.linspace(min(min(theta1), min(theta4)), max(max(theta1), max(theta4)), 2000)

new_datas = []
for theta, data in zip([theta1, theta4], [dataslice1, dataslice4]):
    nd = []
    for row in data:
        interper = interp1d(theta, row, fill_value=np.nan, bounds_error=False)
        nd.append(interper(new_theta))
    new_datas.append(np.array(nd))
    
    # 3

left = np.min(theta1[dataslice1[100] > 0.01])
right = np.max(theta4[dataslice4[100] > 0.01])

overlap_indices = np.where(new_theta>left)[0][0], np.where(new_theta<right)[0][-1]  # indices over which the datasets will overlap

# 4

w1 = np.linspace(0, 1, overlap_indices[1]-overlap_indices[0])
w4 = np.flip(w1)

# 5

overlap1 = w1 * new_datas[0][:, overlap_indices[0]:overlap_indices[1]]
overlap4 = w4 * new_datas[1][:, overlap_indices[0]:overlap_indices[1]]

overlap = overlap1 + overlap4

new_data = np.concatenate((new_datas[1][:, :overlap_indices[0]], overlap, new_datas[0][:, overlap_indices[1]:]), axis=-1)

fig, ax = plotting_functions.plot_2D_mpl(x=new_theta, y=yaxis1, data=new_data, xlabel='kx', ylabel='E_k', title=f'', cmap='Blues')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Au\FS\Au111_FS.png')
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\Au111_FS_EF.png')

x, y, d = new_theta, yaxis1, new_data

In [ ]:
# # Save to .itx (igor) file
# save_to_igor_itx('test.itx', [x], [d], ["cones"], [y])
# fix_itx_format('test.itx')  # fix itx formatting for denoising website

In [ ]:
# Denoised Data
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Au\FS'
fn_dn = 'Au111_FS_EF_dn.itx'

# title = re.search('FS_(.*?)\_', fn_dn).group()
# title = title[3:-1]
title = 'EF'

x, y, data_dn = load_denoised_data(fp_dn, fn_dn)

d = analysis_functions.norm_data(data_dn)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=d, xlabel='kx', ylabel='ky', cmap='Blues')

plotting_functions.format_FS_plot(fig, ax, EF=16.8, title=title, xticks=None, set_aspect_ratio=False)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\Au111_FS_EF_dn.png')

In [ ]:
fn = r'Au111_SS_split_g_kw.ibw'
xaxis, yaxis, data = kw_data_loader.load2D_k(ddir=fp, fn=fn, summary=False);

# ax.set_ylim(-0.55, 0.0)
# plt.yticks(np.arange(-0.5, 0.01, 0.25), fontsize=14)

# ax.set_xlim(-0.3, 0.3)
# plt.xticks(np.arange(-0.25, 0.3, 0.25), fontsize=14)

x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, data, xlim=(-0.3, 0.3), ylim=(-0.55, 0.0), EF=16.9)
normed_data = analysis_functions.norm_data(d)

fig, ax = plotting_functions.plot_2D_mpl(x, y, normed_data, EF=16.9, title='Au(111) Surface State', xlabel='kx (A-1)', ylabel='E-EF (eV)', cmap='Blues')

# set aspect ratio
ratio = 1.0
x_left, x_right = ax.get_xlim()
y_low, y_high = ax.get_ylim()
ax.set_aspect(abs((x_right-x_left)/(y_low-y_high))*ratio)

In [ ]:
new_data = ff.fft2d_mask(data, plot=False)

x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, new_data, xlim=(-0.3, 0.3), ylim=(-0.55, 0.0), EF=16.9)
normed_data = analysis_functions.norm_data(d)

fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, EF=16.9, xlabel='kx', ylabel='E-EF (eV)', title=f'Au(111) Surface State', cmap='Blues')

# set aspect ratio
ratio = 1.0
x_left, x_right = ax.get_xlim()
y_low, y_high = ax.get_ylim()
ax.set_aspect(abs((x_right-x_left)/(y_low-y_high))*ratio)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Denoised Data\Au111_ss.png')

In [ ]:
x.shape, y.shape, normed_data.shape

In [ ]:
# # Save to .itx (igor) file
# d = normed_data
# save_to_igor_itx('test.itx', [x], [d], ["cones"], [y])
# fix_itx_format('test.itx')  # fix itx formatting for denoising website

# Scratch Work

Ideas:
- show how it looks on a really nice dataset (my Au); shouldn't change it much
- show how it really breaks if it doesn't work properly (like if you try to do it to too high an accuracy with too low of statistics)
- compare to Gaussian smoothing
- compare FS vs KE cuts
- remember to change downloaded file to .itx file
- write a mini thesis chapter on DAE
- prepare a presentation for LAIR and David's group on DAE

In [ ]:
# with open('test.itx', 'r') as f:
#     lines = f.readlines()
# #     for i in range(4):
# #         lines.append(f.readline())

# for ln in [1, -3, -2, -1]:
#     lines[ln] = lines[ln].replace("'", "")

    
# new = lines[:-6]
# new[1] = 'WAVES/D/N='+lines[1][12:]
# new.append('END\n')

# dscale = lines[-3][2:-1].replace(',', ' ', 1)
# xscale = lines[-2][2:-1].replace(',', ' ', 1).replace(' ', '', 1)
# yscale = lines[-1][2:-1].replace(',', ' ', 1).replace(' ', '', 1)

# new.append(f'X {xscale}; {yscale}; {dscale}'.replace('"",', '"", '))

# with open('test_fixed.itx', 'w') as f:
#     f.writelines(new)

In [ ]:
# new[-1]

In [ ]:
# print('\'X SetScale/P x -0.399598565297964,0.00185014925677925,"", cones; SetScale/P y 17.5510022132154,0.00292671945009459,"", cones; SetScale d 0,0,"", cones')

In [ ]:
# Try to open the .ibw file created by our .itx file (that can be processed by the denoiser, but isn't generated by the denoiser). I created cones.ibw by loading the .itx file in igor and then saving it as a .ibw file. The .itx file generated by the denoiser can't even be opened in igor...

# Filepath
fp = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\FS'
fn = 'XUV_FS_EF_dn.itx'

# Load .itx formatted data
data_path = os.path.join(fp, fn)
# df = pd.read_csv(data_path, header=2, sep='\s+')  # first 3 lines are comments 
df = pd.read_csv(data_path, skiprows=3, header=None, sep='\s+', skipfooter=2)  # skiprows=3 to skip over the Igor junk, header=None because there are no column names, sep '\s+' for white space separated, skipfooter=2 to skip Igor junk
df
# df2['AB_split'] = df2[0:].str.split('  ')

In [ ]:
data = np.array(df).T
data, data.shape

In [ ]:
import re
with open(data_path, 'r') as f:
    lines = f.readlines()
last = lines[-1]
last

x_start, x_step = [float(v) for v in re.search('x\s*(-?\d*.\d+),\s*(-?\d+.\d+)', last).groups()]
y_start, y_step = [float(v) for v in re.search('y\s*(-?\d*.\d+),\s*(-?\d+.\d+)', last).groups()]

x = np.linspace(x_start, x_start+data.shape[1]*x_step, data.shape[1])
y = np.linspace(y_start, y_start+data.shape[0]*y_step, data.shape[0])
x.shape, y.shape

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Heatmap(x=x, y=y, z=data))
fig.update_traces(showscale=False)
fig.update_layout(height=500, width=600)
fig.show()

# For Jiabin

In [ ]:
!pip install kaleido

from arpes_functions import cnn, tr_functions
import plotly.graph_objects as go

In [ ]:
fp= r'C:\Users\atully\OneDrive\Physics.UBC\Colleagues\CNN'
# fn = r'IMG0001_kw_EB.h5'
fn = r'UPS0001_kw_EB.ibw'
theta, energy, data = kw_data_loader.load2D_k(fp, fn)  # load data from hdf5

In [ ]:
data.shape, theta.shape, energy.shape

In [ ]:
# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=theta, y=energy, data=data, title=f'NbIrTe4', xlabel='kx (A-1)', cmap='Blues', swap_axes=False)

In [ ]:
fn = r'IMG0001_kw_EB.ibw'
theta, energy, data = kw_data_loader.load2D_k(fp, fn)  # load data from hdf5

In [ ]:
# # plot data
# fig, ax = plotting_functions.plot_2D_mpl(x=theta, y=energy, data=data, title=f'NbIrTe4', xlabel='k (A-1)', cmap='Blues', swap_axes=False)
# fig.show()

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\Colleagues\CNN\NbIrTe4.png')

# Plot Data
fig = tr_functions.default_fig()
fig.add_trace(
    go.Heatmap(x=theta, y=energy, z=data, coloraxis="coloraxis")
    # np.log(data)
)
fig.update_coloraxes(colorscale="Blues", showscale=True)

fig.update_layout(xaxis_range=(-0.65, 0.6), height=600, width=500)

fig.update_layout(
    title=f"NbIrTe4",
    xaxis_title="k (A-1)",
    yaxis_title="energy (eV)",
)
fig.show()

fig.write_image(r'C:/Users/atully/OneDrive/Physics.UBC/Colleagues/CNN/NbIrTe4.png')

# # Save to .itx (igor) file
# x, y, d = theta, energy, data
# cnn.save_to_igor_itx('test.itx', [x], [d], ["Jiabin"], [y])  # take every other y value to make dataset smaller
# cnn.fix_itx_format('test.itx')  # fix itx formatting for denoising website

In [ ]:
d, x, y = analysis_functions.get_data_region(data, theta, energy, xbounds=(-0.25, 0.25), ybounds=(-0.2, 0.05))

x.shape, y.shape, d.shape

In [ ]:
# # plot slice
# fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=d, title=f'NbIrTe4', xlabel='kx (A-1)', cmap='Blues', swap_axes=False)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\Colleagues\CNN\NbIrTe4_inset.png')

# Plot Data
fig = tr_functions.default_fig()
fig.add_trace(
    go.Heatmap(x=x, y=y, z=d, coloraxis="coloraxis")
    # np.log(data)
)
fig.update_coloraxes(colorscale="Blues", showscale=True)

fig.update_layout(height=600, width=500)

fig.update_layout(
    title=f"NbIrTe4 Inset",
    xaxis_title="k (A-1)",
    yaxis_title="Binding Energy (eV)",
)
fig.show()

fig.write_image(r'C:/Users/atully/OneDrive/Physics.UBC/Colleagues/CNN/NbIrTe4_inset_v2.png')

# # Save to .itx (igor) file
# x, y, d = x, y, d
# cnn.save_to_igor_itx('test.itx', [x], [d], ["Jiabin"], [y])  # take every other y value to make dataset smaller
# cnn.fix_itx_format('test.itx')  # fix itx formatting for denoising website

In [ ]:
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\Colleagues\CNN'
fn_dn = 'NbIrTe4_dn.itx'

x_dn, y_dn, data_dn = load_denoised_data(fp_dn, fn_dn)

# Plot Data
fig = tr_functions.default_fig()
fig.add_trace(
    go.Heatmap(x=x_dn, y=y_dn, z=data_dn, coloraxis="coloraxis")
    # np.log(data)
)
fig.update_coloraxes(colorscale="Blues", showscale=True)

fig.update_layout(xaxis_range=(-0.65, 0.6), height=600, width=500)

fig.update_layout(
    title=f"NbIrTe4 Denoised",
    xaxis_title="k (A-1)",
    yaxis_title="Binding Energy (eV)",
)

fig.show()

fig.write_image(r'C:/Users/atully/OneDrive/Physics.UBC/Colleagues/CNN/NbIrTe4_dn.png')

In [ ]:
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\Colleagues\CNN'
# fn_dn = 'NbIrTe4_inset_dn.itx'
fn_dn = 'NbIrTe4_inset_v2_dn.itx'

x_dn, y_dn, data_dn = load_denoised_data(fp_dn, fn_dn)

# Plot Data
fig = tr_functions.default_fig()
fig.add_trace(
    go.Heatmap(x=x_dn, y=y_dn, z=data_dn, coloraxis="coloraxis")
    # np.log(data)
)
fig.update_coloraxes(colorscale="Blues", showscale=True)

fig.update_layout(height=600, width=500)

fig.update_layout(
    title=f"NbIrTe4 Inset Denoised",
    xaxis_title="k (A-1)",
    yaxis_title="Binding Energy (eV)",
)

fig.show()

fig.write_image(r'C:/Users/atully/OneDrive/Physics.UBC/Colleagues/CNN/NbIrTe4_inset_v2_dn.png')

In [ ]:
!pip install kaleido